Hello, Welcome to the Internal Guide to this Uncertainy Library-- This python notebook should contain everything a man needs to use and alter this package as he sees fit.

Throughout this, we are going to use the same datasets for regression and classification. The regression dataset can be found at https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip and the classifiction dataset can be found at https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz. You will obviously need to get these to continue this notebook

## NOTE: HAD to change the extention of covtype to .txt


This library has a dependency on this repo here: https://github.com/cpmpercussion/keras-mdn-layer. Shoutout to compercussion on github for making the mdn layer, which is extremely good.

# NOTE:  python3 -m pip install keras-mdn-layer

Now, you might note: the scores here in the default training are really bad-- Train for more epochs! This is a demo of how to use stuff.

In [1]:
import tensorflow as tf
import WUT as WUT
import numpy as np
tf.__version__

'2.4.1'

In [2]:
'Here we make a Regression Dataset'

text_file = open("Datasets/YearPredictionMSD.txt", "r")
lines = text_file.readlines()
alldata = [[float(x) for x in line.split(',')] for line in lines]
text_file.close()
alldata = np.array(alldata)
Yreg = tf.convert_to_tensor(alldata[:,0], dtype = tf.float32)
Xreg = tf.convert_to_tensor(alldata[:,1:], dtype = tf.float32)
# Yreg = (Yreg - tf.math.reduce_mean(Yreg))/tf.math.reduce_std(Yreg)
# Xreg = (Xreg - tf.math.reduce_mean(Xreg, axis=0, keepdims=True))/tf.math.reduce_std(Xreg, axis=0, keepdims=True)



# Yreg = Yreg/tf.reduce_max(Yreg)

print(Xreg.shape, Yreg.shape)

'Here we make a classification Dataset'


# text_file = open("Datasets/covtype.txt", "r")
# lines = text_file.readlines()
# alldata = [[float(x) for x in line.split(',')] for line in lines]
# text_file.close()
# alldata = np.array(alldata)
# ImYclass = alldata[:,-1].astype(int)-1
# Xclass = (alldata[:,:-1])
# Yclass = np.zeros((ImYclass.size, ImYclass.max()+1))
# Yclass[np.arange(ImYclass.size), ImYclass] = 1
# Yclass = tf.convert_to_tensor(Yclass, dtype = tf.float32)
# Xclass = tf.convert_to_tensor(Xclass, dtype = tf.float32)

# print(Xclass.shape, Yclass.shape)





(515345, 90) (515345,)
(581012, 54) (581012, 7)


Now we're gonna make two models, very simple bois for regression and classification, respectively. IMPORTANT NOTE that in these networks, the output to the call is the output of the last Dense Layer. This is the only requirement for this package to work with your models. (Ensembles and Dropout should work for anything, though)

In [7]:

class RegModel(tf.keras.Model):

    def __init__(self):
        super(RegModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(60, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
class ClassModel(tf.keras.Model):

    def __init__(self):
        super(ClassModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(36, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(36, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(7, activation = tf.nn.softmax)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)

        x = self.dense2(x)
        return self.dense3(x)
    


You could then train these guys normally: 

In [4]:
regmodel = RegModel()
regmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.MSE)
regmodel.fit(Xreg, Yreg)
print(regmodel.evaluate(Xreg[0:5], Yreg[0:5]))
print(regmodel.predict(Xreg[0:5]))


# classmodel = ClassModel()
# classmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy())
# classmodel.fit(Xclass, Yclass)
# print(classmodel.evaluate(Xclass[0:5], Yclass[0:5]))
# print(classmodel.predict(Xclass[0:5]))

1/1 [==============================] - 0s 64ms/step - loss: 10871.5703
10871.5703125
[[2033.3293]
 [2151.5332]
 [2091.0142]
 [1992.1365]
 [2150.904 ]]


Alternatively, you could wrap them in an ensemble first 
(that second argument in the wrapper is the number of ensembles you'd like). 

The output will now be a mean and variance of ensemble outputs.

# NOTE: Fails with tf 2.1 No module named 'tensorflow.python.keras.layers.ops'" Needs to upgrade to >=2.2. Currently, testing using 2.4.1

In [5]:
from WUT.Ensemble import Ensemble

regmodel = Ensemble(RegModel, 2)
regmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.MSE)
regmodel.fit(Xreg, Yreg)
print(regmodel.evaluate(Xreg[0:5], Yreg[0:5]))
print(regmodel.predict(Xreg[0:5]))





# classmodel = Ensemble(ClassModel, 2)
# classmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy())
# classmodel.fit(Xclass, Yclass)
# print(classmodel.evaluate(Xclass[0:5], Yclass[0:5]))
# print(classmodel.predict(Xclass[0:5]))




1/1 [==============================] - 0s 68ms/step - loss: 2475.5317
[9296.6826171875, 2475.53173828125]
tf.Tensor(
[[[2031.4398  ]
  [  24.871704]]

 [[2050.9312  ]
  [  39.771606]]

 [[2055.4097  ]
  [  43.64148 ]]

 [[1978.0267  ]
  [  31.785034]]

 [[2128.9067  ]
  [  32.361084]]], shape=(5, 2, 1), dtype=float32)


Or, you could wrap them in an MC dropout scheme first 

(second arg is dropout rate, 

third arg defaults to what is below, and determines what types of layers dropout is applied to-- this hasn't been tested on everything, but Dense layers are definitely safe)

The output will now be a mean and variance of multiple trials through the network, with trials obviously determining trials.

In [6]:
from WUT.Dropout import Dropout

regmodel = Dropout(RegModel, 0.2, dropout_layers = [tf.keras.layers.Dense])
regmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.MSE)
regmodel.fit(Xreg, Yreg)
print(regmodel.evaluate(Xreg[0:5], Yreg[0:5]) )
print(regmodel.predict(Xreg[0:5], trials = 2))







classmodel = Dropout(ClassModel, 0.2)
classmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy())
classmodel.fit(Xclass, Yclass)
print(classmodel.evaluate(Xclass[0:5], Yclass[0:5]))
print(classmodel.predict(Xclass[0:5], trials = 2) )






1/1 [==============================] - 0s 9ms/step - loss: 157584.3438
[43890.1796875, 57495.89453125, 157584.34375]
tf.Tensor(
[[[1718.5996  ]
  [ 473.48505 ]]

 [[2324.478   ]
  [ 103.155396]]

 [[1930.6385  ]
  [ 106.90381 ]]

 [[2000.2935  ]
  [ 118.76782 ]]

 [[2250.0103  ]
  [ 216.67645 ]]], shape=(5, 2, 1), dtype=float32)
1/1 [==============================] - 0s 10ms/step - loss: 3.1505
[2.5753939151763916, 2.4598770141601562, 3.150456190109253]
tf.Tensor(
[[[2.3829305e-01 5.9207106e-01 9.7912557e-02 7.5984714e-03 1.4000494e-02
   4.4260804e-02 5.8635855e-03]
  [7.7789672e-02 8.2211971e-02 9.7684808e-02 7.5439024e-03 9.4821285e-03
   4.4090934e-02 1.1998494e-03]]

 [[3.1841362e-01 6.7198694e-01 2.6341496e-04 5.3508898e-05 4.4400590e-03
   2.0821295e-04 4.6342975e-03]
  [4.4009984e-03 1.4547406e-03 1.9797857e-04 3.2173601e-05 1.4965509e-03
   1.5893567e-04 1.0606315e-03]]

 [[1.9156094e-01 7.6364744e-01 1.0298616e-02 4.3134470e-04 1.2862938e-02
   1.2706689e-02 8.4920758e-03]
  

Or, you could simply throw an additional network to predict error, as a regression

(extra args:

Std_Model: The model to predict variance. Defaults to a clone of input model, with activation on last layer removed)

error_activation: What to replace the activation on last layer with for the Std_Model. Defaults to None

one_hot: If true, will assume target values have already been converted to one_hot variables, and will not do it manually. Defaults to True.)


The output will now be simple predicted mean and variance.

In [7]:
from WUT.Variance_Network import Variance_Network

regmodel = Variance_Network(RegModel)
regmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.MSE)
regmodel.fit(Xreg, Yreg)
print(regmodel.evaluate(Xreg, Yreg)) 
print(regmodel.predict(Xreg[0:5]))






classmodel = Variance_Network(ClassModel)
classmodel.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy())
classmodel.fit(Xclass, Yclass)
print(classmodel.evaluate(Xclass, Yclass))
print(classmodel.predict(Xclass[0:5]))






TypeError: function() got an unexpected keyword argument 'jit_compile'

Or, you could fit a gaussian mixture model to this.
Second argument here is how many gaussians you'd like to fit.

Output is now a Probability distribution in posterior space


In [8]:
from WUT.Gaussian_Mixtures import Gaussian_Mixtures

regmodel = Gaussian_Mixtures(RegModel, 5)
regmodel.compile(optimizer = tf.keras.optimizers.Adam())
regmodel.fit(Xreg, Yreg)
print(regmodel.evaluate(Xreg, Yreg)) 
regression_dist = regmodel.predict(Xreg[0:5])
print(regression_dist.sample())






classmodel = Gaussian_Mixtures(ClassModel, 5)
classmodel.compile(optimizer = tf.keras.optimizers.Adam())
classmodel.fit(Xclass[0:5], Yclass[0:5], epochs = 1000, verbose = False)
print(classmodel.evaluate(Xclass[0:5], Yclass[0:5]))
classification_dist = classmodel.predict(Xclass[0:5])
print(classification_dist.sample())


Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
16105/16105 [==============================] - 11s 701us/step - loss: 6.2689
6.268917560577393
tf.Tensor(
[[1921.5614]
 [2114.6772]
 [2088.1204]
 [2074.249 ]
 [2280.48  ]], shape=(5, 1), dtype=float32)
1/1 [==============================] - 0s 990us/step - loss: 21114667008.0000
21114667008.0
tf.Tensor(
[[  -2.8682199   440.55505      -9.252266   -122.47605     285.3768
  -189.76274     -86.552826  ]
 [   0.63337547  485.99738    -286.06485    -224.61842    -326.705
    44.933136     -9.93264   ]
 [   1.9110353   292.4095      -83.9474        5.6412625   163.41637
   108.41966    -209.20148   ]
 [  -1.9658955   143.63368     -38.92147     -30.871521      7.049843
  -156.00131    -307.8017    ]
 [   2.242136   -328.57556    -191.11871      36.756676    164.90022
   137.1727      -38.892155  ]], shape=(5, 7), dtype=float32)


Lastly, if you've got a real masochistic streak, you could run Stochastic Variational Inference Here.

There are a lot of optional arguments here for those who want to get into the weeds, but I'll cover a few important ones:

task: set this to 'regression' or 'classification'. Defaults to regression.

one_hot: If True, assumes targets are one hot in classification case. Defaults True

normalize: whether to normalize inputs and outputs, defaults to True. If you encounter nans, try fiddling with this!

Returns: A probability distribution over the outputs pace



In [8]:
from WUT.SVI import SVI

regmodel = SVI(RegModel)
regmodel.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001))
regmodel.fit(Xreg, Yreg)
print(regmodel.evaluate(Xreg, Yreg)) 
regression_dist = regmodel.predict(Xreg[0:5])
print(regression_dist.sample())






# classmodel = SVI(ClassModel, task = 'classification', one_hot = True)
# classmodel.compile(optimizer = tf.keras.optimizers.Adam())
# classmodel.fit(Xclass, Yclass)
# print(classmodel.evaluate(Xclass, Yclass))
# classification_dist = classmodel.predict(Xclass[0:5])
# print(classification_dist.sample())



16105/16105 [==============================] - 9s 574us/step - loss: 7941.6631
7941.6630859375
tf.Tensor(
[[2037.2229]
 [1959.3247]
 [1946.5162]
 [1973.3037]
 [2035.2181]], shape=(5, 1), dtype=float32)
